In [101]:
import numpy as np
import random
from matplotlib import pyplot as plt

# Multi Armed Bandit Problem

<img src="epsilongreedy.png">

In [130]:
class bandit_urn:
    #Inicialization
    # levers: integer, number of levers
    # real_distribution: list, a list containing the ground truth probability of recieving a reward respectively, in this case of observing a crime
    # observed: list, a list containing the observed probability of recieving a reward respectively 
    # observed_size: list, a list of many times has each lever been pulled respectively
    # epsilon: float, a number between 0 and 1.
    # urn_balls: list of balls asociated to each neighborhood
    # lambdas: list of likelihood of a crime happening (probability or )
    # tot_reward: int, historic reward
    def __init__(self, real_distribution, epsilon, urn_balls, tot_reward=0):
        self.real_distribution=real_distribution
        self.epsilon=epsilon
        self.urn_balls=urn_balls
        self.tot_reward=tot_reward
    
    def total_balls(self):
        return(sum(self.urn_balls))
    
    # Calculate ratios of balls
    def ball_rates(self):
        rates=[i/self.total_balls() for i in self.urn_balls]
        return(rates)
    
    # Choose the neighborhood
    def choose_greedy(self):
        index=np.random.choice(len(self.urn_balls), p=self.ball_rates())
        return(index)
    
    def observe_reward(self,num_lever):
        reward=self.real_distribution[num_lever]
        return(reward)
 
    # Choose between exploration or explotation depending of epsilon probability
    def explore_or_exploit(self):
        explore=np.random.uniform(0,1)<self.epsilon
        return(explore)
    
    # The corresponding entry of the observed and observed_size lists are updated
    def update_observed(self, num_lever, reward):
        self.urn_balls[num_lever]=(self.urn_balls[num_lever]+reward)
        
    # Epsilon Greedy model
    def choose_e_greedy(self):
        action=self.explore_or_exploit()
        if action:
            which=random.randint(0,(len(self.urn_balls)-1))
            reward=self.observe_reward(which)
            self.tot_reward=self.tot_reward+reward
            self.update_observed(which, reward)
        else:
            which=self.choose_greedy()
            reward=self.observe_reward(which)
            self.tot_reward=self.tot_reward+reward
            self.update_observed(which, reward)
    
    # Simulate N periods of Epsilon-greedy algorithm
    def N_times_e_greedy(self, periods):
        observed_history=[]
        observed_history.append(self.ball_rates())
        for i in range(periods):
            self.choose_e_greedy()
            observed_history.append(self.ball_rates())
        return(observed_history)

#### Parameters

In [184]:
num_levers=3 # number of levers.
real=[3, 2, 5] # real distribution of rewards.
obs=[100, 100, 100] # observed distribution of rewards.
eps=0.9 # Probability (trade-off) of exploring instead of exploiting.
tot=0 # Starting reward.
N=10000 # periods of simulation.
k=100 # number of simulations

In [185]:
prueba=bandit_urn(real_distribution=real, urn_balls=obs, epsilon=eps)

In [186]:
prueba.ball_rates()

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [187]:
%%time
history=prueba.N_times_e_greedy(1000)

Wall time: 9.97 ms


In [188]:
dist_real=[i/sum(prueba.real_distribution) for i in prueba.real_distribution]

In [189]:
history[1]

[0.33112582781456956, 0.33774834437086093, 0.33112582781456956]

In [190]:
red=[i[0] for i in history]
blue=[i[1] for i in history]
green=[i[2] for i in history]

In [191]:
plt.plot(red, color='red')
plt.plot(blue, color='blue')
plt.plot(green, color='green')

plt.plot(range(len(red)), [dist_real[0]]*len(red), color='red', linestyle='--')
plt.plot(range(len(red)), [dist_real[1]]*len(red), color='blue', linestyle='--')
plt.plot(range(len(red)), [dist_real[2]]*len(red), color='green', linestyle='--')

In [34]:
%%time
# K Simulations of N periods.

history=[]
history_exp=[]
history_obs=[]
for i in range(k):
    test=bandit_urn(levers=num_levers, real_distribution=real, observed=obs, observed_size=size , epsilon=eps, tot_reward=tot)
    vector, vector_exp, observed=test.N_times_e_greedy(N)
    history.append(vector)
    history_exp.append(vector_exp)
    history_obs.append(observed)
    obs=[0.5, 0.3, 0.2] # observed distribution of rewards.
    size=[12, 15, 18] # number of times each levers is pulled.

Wall time: 4.49 s


In [35]:
obs_mean_hist_0=[];obs_mean_hist_1=[];obs_mean_hist_2=[]
obs0=[];obs1=[];obs2=[]
for j in range(N):    
    for i in range(k):    
        obs0.append(history_obs[i][j][0])
        obs1.append(history_obs[i][j][1])
        obs2.append(history_obs[i][j][2])
    obs_mean_hist_0.append(np.mean(obs0))
    obs_mean_hist_1.append(np.mean(obs1))
    obs_mean_hist_2.append(np.mean(obs2))

In [36]:
test.real_distribution

[0.1, 0.2, 0.3]

In [37]:
test.observed

[0.12755598831548204, 0.1987242394504416, 0.29273659207400843]

In [38]:
N

10000

In [39]:
plt.figure(figsize=(8,8))
plt.plot(obs_mean_hist_0, color='blue')
plt.plot((range(N)),[test.real_distribution[0]]*N, color='blue', linestyle='--')
plt.plot(obs_mean_hist_2, color='red')
plt.plot(range(N),[test.real_distribution[2]]*N, color='red', linestyle='--')
plt.plot(obs_mean_hist_1, color='orange')
plt.plot(range(N),[test.real_distribution[1]]*N, color='orange', linestyle='--')

In [40]:
# Computate mean and standard deviation for confidence intervals
def build_mean_CI(list_of_lists):
    means=[]
    std=[]
    for j in range(len(list_of_lists[0])):
        values=[]
        for i in list_of_lists:
            values.append(i[j])
        means.append(np.mean(values))
        std.append(np.std(values))
    return(means, std)

In [41]:
means_exp, std_exp=build_mean_CI(history_exp)
means, std=build_mean_CI(history)

In [141]:
len(means_exp)

1000

In [42]:
plt.figure(figsize=(12,6))
plt.plot(range(len(means)), means_exp)
plt.plot(range(len(means)), [max(test.real_distribution)]*len(means), color='red')
#plt.plot(range(len(std)), )

In [ ]:
len(history_exp)

In [ ]:
expected_reward=[rewards[i]/(range(len(rewards))[i]+1) for i in range(len(rewards))]

In [ ]:
plt.plot(range(len(rewards)), expected_reward)